# Hybrid Encryption

In [ ]:
!pip3 install pycrypto stegano pycryptodome black blackcellmagic

In [2]:
%load_ext blackcellmagic

## Encryption

In [15]:
from Crypto.Cipher import Blowfish, PKCS1_OAEP, AES
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad, unpad
from binascii import hexlify, unhexlify
import hashlib
import json
import string
import random
from stegano import lsb
from datetime import datetime

### Key Generator for AES and Blowfish

This generates random keys based on three parameters

- size: length of the key to generate
- case: casing of ASCII letters, possible values: "default", "upper-case-only", "upper-case-only"
- punctuations: if punctuation symbols are required in  !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~


In [16]:
def key_generator(size: int, case: str = "default", punctuations: bool = True):
    if punctuations:
        chars = string.ascii_letters + string.digits + string.punctuation
    else:
        chars = string.ascii_letters + string.digits

    if case == "upper-case-only":
        chars = chars.upper()
    elif case == "lower-case-only":
        chars = chars.lower()

    if size > len(chars):
        raise ValueError("Key size is larger than the available characters")

    return ''.join(random.choice(chars) for _ in range(size))

### Blowfish Algorithm



In [17]:
# Blowfish
def blowfish_encrypt(plaintext):
    blowfish_key = key_generator(size=16).encode()
    blowfish_cipher = Blowfish.new(blowfish_key, Blowfish.MODE_CBC)

    blowfish_ciphertext = blowfish_cipher.encrypt(
        pad(plaintext, Blowfish.block_size))

    return blowfish_key, blowfish_cipher.iv, blowfish_ciphertext

In [18]:
# RSA
def rsa_encrypt(plaintext):
    rsa_key = RSA.generate(2048)
    rsa_private_key = rsa_key
    rsa_public_key = rsa_key.publickey()

    cipher_rsa = PKCS1_OAEP.new(rsa_public_key)

    rsa_ciphertext = bytearray()
    for i in range(0, len(plaintext), 190):
        rsa_ciphertext.extend(cipher_rsa.encrypt(plaintext[i: i + 190]))

    return rsa_private_key.n, rsa_private_key.e, rsa_private_key.d, rsa_ciphertext

In [19]:
# AES
def aes_encrypt(plaintext):
    aes_key = key_generator(size=16).encode()
    aes_cipher = AES.new(aes_key, AES.MODE_CBC)
    aes_ciphertext = aes_cipher.encrypt(pad(plaintext, AES.block_size))

    return aes_key, aes_cipher.iv, aes_ciphertext

In [20]:
# LSB Steg
def lsb_steganography(encrypted_keys_and_iv):
    lsb_stegano_image = lsb.hide(
        "./cover_image.png", encrypted_keys_and_iv.decode())
    lsb_stegano_image.save("./stego_image.png")

In [21]:
def encrypt_keys(password, keys_iv):
    hash = hashlib.sha1()
    hash.update(password.encode())
    password_encryption_cipher = AES.new(
        hash.hexdigest()[:16].encode(), AES.MODE_CBC, iv="16bitAESInitVect".encode()
    )
    # Encryption of Key and IV String
    encrypted_keys_and_iv = hexlify(
        password_encryption_cipher.encrypt(
            pad(json.dumps(keys_iv).encode(), AES.block_size)
        )
    )
    return encrypted_keys_and_iv

In [22]:
def open_file(path: str) -> bytes:
    with open(path, "rb") as file:
        plaintext = file.read()
    return plaintext


def output_file(path, ciphertext):
    output_filepath = path + "_hyenc.encrypted"
    with open(output_filepath, "w") as file:
        file.write(hexlify(ciphertext).decode())

In [23]:
# Run Encryption
def run_encryption(plaintext):
    keys_iv = {}

    blowfish_key, blowfish_iv, blowfish_ct = blowfish_encrypt(plaintext)
    keys_iv["blowfish_iv"], keys_iv["blowfish_key"] = (
        hexlify(blowfish_iv).decode(),
        hexlify(blowfish_key).decode(),
    )

    rsa_n, rsa_e, rsa_d, rsa_ct = rsa_encrypt(blowfish_ct)
    keys_iv["rsa_n"], keys_iv["rsa_e"], keys_iv["rsa_d"] = rsa_n, rsa_e, rsa_d

    aes_key, aes_iv, aes_ct = aes_encrypt(rsa_ct)
    keys_iv["aes_iv"], keys_iv["aes_key"] = (
        hexlify(aes_iv).decode(),
        hexlify(aes_key).decode(),
    )

    return keys_iv, aes_ct

In [24]:
input_file_path = input("Enter Filepath of file to encrypt: ")

plaintext = open_file(input_file_path)

log_plaintext_length = len(hexlify(plaintext))

# Password for Key Encryption
password = input("Enter Password: ")
log_password_length = len(password)

log_start_time = datetime.now()

keys_iv, ciphertext = run_encryption(plaintext)

output_file(input_file_path, ciphertext)

encrypted_keys_and_iv = encrypt_keys(password, keys_iv)
lsb_steganography(encrypted_keys_and_iv)

log_ciphertext_length = len(hexlify(ciphertext))
log_end_time = datetime.now()
log_duration = str(log_end_time - log_start_time)

with open("./logs/encryption-log.txt", "a+") as log_file:
    log_file.write(
        "\n| "
        + str(log_plaintext_length)
        + "          | "
        + str(log_ciphertext_length)
        + "          | "
        + str(log_password_length)
        + "         | "
        + log_start_time.strftime("%H:%M:%S")
        + "   | "
        + log_end_time.strftime("%H:%M:%S")
        + "  | "
        + str(log_duration)
        + " |"
    )

print("File Encryption Complete!")

File Encryption Complete!


## Decryption

In [25]:
def get_encrypted_file(file_to_decrypt):
    with open(file_to_decrypt, 'r') as file:
        ciphertext = file.read()

    return ciphertext

In [26]:
def get_key_iv_from_steg_image(password):
    unhide_encrypted_keys_and_iv = lsb.reveal("./stego_image.png").encode()
    hash = hashlib.sha1()
    hash.update(password.encode())
    password_decryption_cipher = AES.new(
        hash.hexdigest()[:16].encode(), AES.MODE_CBC, iv='16bitAESInitVect'.encode())

    return json.loads(unpad(password_decryption_cipher.decrypt(unhexlify(unhide_encrypted_keys_and_iv)), AES.block_size))

In [27]:
def aes_decrypt(ciphertext, decrypted_keys_iv):
    decryption_key_aes = unhexlify(decrypted_keys_iv['aes_key'])
    decryption_iv_aes = unhexlify(decrypted_keys_iv['aes_iv'])
    aes_cipher_decryption = AES.new(
        decryption_key_aes, AES.MODE_CBC, iv=decryption_iv_aes)
    return unpad(aes_cipher_decryption.decrypt(unhexlify(ciphertext)), AES.block_size)

In [28]:
def rsa_decrypt(ciphertext, decrypted_keys_iv):
    decryption_key_rsa = RSA.construct(rsa_components=(
        decrypted_keys_iv['rsa_n'], decrypted_keys_iv['rsa_e'], decrypted_keys_iv['rsa_d']))
    rsa_cipher_decryption = PKCS1_OAEP.new(decryption_key_rsa)
    ciphertext_blowfish = bytearray()
    for i in range(0, len(ciphertext), 256):
        ciphertext_segment = ciphertext[i:i+256]
        ciphertext_blowfish.extend(
            rsa_cipher_decryption.decrypt(ciphertext_segment))
    return ciphertext_blowfish

In [29]:
def blowfish_decrypt(ciphertext, decrypted_keys_iv):
    decryption_iv_blowfish = unhexlify(decrypted_keys_iv['blowfish_iv'])
    decryption_key_blowfish = unhexlify(decrypted_keys_iv['blowfish_key'])
    blowfish_cipher_decryption = Blowfish.new(
        decryption_key_blowfish, Blowfish.MODE_CBC, iv=decryption_iv_blowfish)
    return unpad(blowfish_cipher_decryption.decrypt(ciphertext), Blowfish.block_size)

In [30]:
def create_decrypted_output_file(filepath, decrypted_plaintext):
    output_filepath = filepath.replace('_hyenc.encrypted', '')
    output_folder_path = "./dec_"+output_filepath.split("/")[-1]
    with open(output_folder_path, 'wb+') as file:
        file.write(decrypted_plaintext)
        file.close()
    return output_folder_path

In [31]:
# Run Encryption
def run_decryption(ciphertext, keys_iv):
    aes_plaintext = aes_decrypt(ciphertext, keys_iv)
    rsa_plaintext = rsa_decrypt(aes_plaintext, keys_iv)
    blowfish_plaintext = blowfish_decrypt(rsa_plaintext, keys_iv)
    return blowfish_plaintext

In [32]:
password = input("Enter Password: ")
file_to_decrypt = input("Enter Filepath of file to decrypt: ")


log_password_length = len(password)
log_ciphertext_length = len(ciphertext)

log_start_time = datetime.now()

decrypted_keys_iv = get_key_iv_from_steg_image(password)
ciphertext = get_encrypted_file(file_to_decrypt)
decrypted_plaintext = run_decryption(ciphertext, decrypted_keys_iv)

output_file_path = create_decrypted_output_file(
    file_to_decrypt, decrypted_plaintext)


log_end_time = datetime.now()
log_duration = str(log_end_time - log_start_time)
log_plaintext_length = len(hexlify(decrypted_plaintext))


with open("./logs/decryption-log.txt", "a+") as log_file:
    log_file.write(
        "\n| "
        + str(log_ciphertext_length)
        + "          | "
        + str(log_plaintext_length)
        + "          | "
        + str(log_password_length)
        + "         | "
        + log_start_time.strftime("%H:%M:%S")
        + "   | "
        + log_end_time.strftime("%H:%M:%S")
        + "  | "
        + str(log_duration)
        + " |"
    )


print("File Decryption Complete! ", output_file_path)

File Decryption Complete!  ./dec_sample.txt
